#Projekt pod tytułem "Titanic" 

https://www.kaggle.com/c/titanic

Projekt polegał na przewidzeniu, kto z pasażerów wymienionego Titanica przeżyje.

Baza "train.csv" zawiera informacje o podróżnych oraz o tym czy przeżyli katastrofę statku. Baza "test.csv" przedstawia listę osób, których przezywalność należy sprawdzić.


Zaimportowanie baz "numpy" i "pandas".

In [23]:
import numpy as np # to linear algebra
import pandas as pd # to data processing

Zaimportowanie bazy danych "train.csv" oraz wypisanie paru pierwszychwierszy.

In [24]:
train_data = pd.read_csv("https://raw.githubusercontent.com/Mateusz337/Missp/main/projekt/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Zaimportowanie bazy danych "test.csv" oraz wypisanie paru pierwszych wierszy.

In [25]:
test_data = pd.read_csv("https://raw.githubusercontent.com/Mateusz337/Missp/main/projekt/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Wypisanie danych podsumowująych z baz "train.csv" oraz "test.csv".

In [26]:
print("Dane train:")
train_data.describe()

Dane train:


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [27]:
print("Dane data:")
test_data.describe()

Dane data:


,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


Sprawdzanie typu danych pod względem kolumn i wypełnionych wierszy w bazach "train.csv" oraz "test.csv".

In [28]:
train_data.info()
print()
print(50*"=")
print()
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass

Jak można zauważyć część wierszy jest pustych, więc należy je uzupełnić lub usunąć. 

In [30]:
print("Przed czyszczeniem:")
print("Brakujące dane w bazie train:")
missing_val_count_by_column = (train_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])
print()
print("Brakujące dane w bazie test:")
missing_val_count_by_column = (test_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Przed czyszczeniem:
Brakujące dane w bazie train:
Age         177
Cabin       687
Embarked      2
dtype: int64

Brakujące dane w bazie test:
Age       86
Fare       1
Cabin    327
dtype: int64


Można zauważyć pare problemów:

*   Brak wieku podróżnych w bazie train.csv oraz test.csv. Dla uśrednienia zostaną one dodane jako mediana wieku pdróżnych.
*   Brak informacji o miejscu podjęcia na statek pasażerów w bazie train.csv. Tych danych nie da się uśrednić czy znaleźć, więc zostaną usunięte.
*   Brak opłaty za bilet w bazie test.csv. Zostanie dodana jako mediana cen biletów.
*   Kolumna "Embarked" w bazie train.csv. Jej typ zostanie zmieniony.

In [32]:
train_data.Age = train_data.Age.fillna(train_data.Age.median())

train_data.dropna(subset=['Embarked'],inplace = True)
one_hot_encoded_training_predictors = pd.get_dummies(train_data.Embarked)

test_data.Age = test_data.Age.fillna(test_data.Age.median())
test_data.Fare = test_data.Fare.fillna(test_data.Fare.median())

Poniżej została pokazana wyłącznie ilość brakujących danych, które są niepotrzebne do dalszej pracy.

In [33]:
print("Po czyszczeniu:")
print("Brakujące dane w bazie train:")
missing_val_count_by_column = (train_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])
print()
print("Brakujące dane w bazie test:")
missing_val_count_by_column = (test_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])


Po czyszczeniu:
Brakujące dane w bazie train:
Cabin    687
dtype: int64

Brakujące dane w bazie test:
Cabin    327
dtype: int64


Model jest uczony metodą losowego lasu decyzyjnego. W celu przewidzenia, który pasażer przeżyje należy skorzystać z danych: 
* wiek 
* płeć
* klasa biletu 
* wysokość opłaty za bilet 
* ilość rodzeństwa na statku
* ilość opiekunów na statku
* port, z którego podjęto pasażera


Ilość drzew to 130, a maksymalna liczba rozgałęzień to 7. 

Dopasowanie będzie równe 0.7919.

In [35]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=130, max_depth=7, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Twoje zgłoszenie zostało pomyślnie zapisane!!!")

Twoje zgłoszenie zostało pomyślnie zapisane!!!
